# Preparação de Dados

## 1 Byte pair encoding de palavras fora do voculário

Durante a aula vimos que um tokenizador baseado em Byte pair encoding (BPE) é capaz de lidar com palavras fora do vocabulário ao dividir uma palavra em "sub-palavras" que estejam presentes no vocabulário. Na pior das hipóteses a palavra pode ser quebrada em letras individuais.

O texto abaixo é um trecho tirado do primeiro parágrafo do livro "The Time Machine" (H. G. Wells, 1895). Use o Tiktoken (com encoding do gpt2) visto durante a aula para tokenizá-lo e verifique quais palavras não estão presentes no vocabulário e necessitaram ser quebradas em "sub-palavras". Mostre como ficou a divisão de cada uma das palavras originalmente fora do vocabulário após a tokenização.

Por exemplo, a palavra "luxurious":<br>
`luxurious -> ['lux', 'urious']`

In [ ]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.8.0
tiktoken version: 0.11.0


In [2]:
time_machine_text = 'The Time Traveller was expounding a recondite matter to us. \
His grey eyes shone and twinkled, and his usually pale face was flushed and animated.'

In [ ]:
import tiktoken

# SEU CÓDIGO AQUI
def Q1():
    tokenizer = tiktoken.get_encoding("gpt2")

    for word in time_machine_text.split():
        encoded_word = tokenizer.encode(word, allowed_special={"<|endoftext|>"})
        decoded_segments = [tokenizer.decode([i]) for i in encoded_word]
        print(f"{word} -> {decoded_segments}")

Q1()

## 2 Data loader com diferentes tamanhos de contexto e strides

Durante a aula, vimos como criar um data loader pra treinar uma LLM através da tarefa de prever o próximo token. No caso, o input `x` é uma sequência de tokens e o alvo `y` é o próximo token da sequência `x`. O data loader cria uma janela deslizante que percorre todo o texto, gerando inúmeros exemplos de treino `x, y`. A quantidade de dados de treino gerada pelo data loader vai variar de acordo com o tamanho de `x` (`max_length`) e o tanto que a janela irá deslizar (`stride`) ao longo do texto.

Use o data loader visto durante a aula para tokenizar o texto abaixo com duas configurações distintas:
- `batch_size=4, max_length=2, stride=1`
- `batch_size=4, max_length=6, stride=2`

E responda, quantos exemplos de treino cada configuração o data loader gerou? Lembre-se que cada batch pode conter até 4 exemplos de treino.

In [ ]:
time_machine_text = "The Time Traveller (for so it will be convenient to speak of him) \
was expounding a recondite matter to us. His grey eyes shone and \
twinkled, and his usually pale face was flushed and animated. The \
fire burned brightly, and the soft radiance of the incandescent \
lights in the lilies of silver caught the bubbles that flashed and \
passed in our glasses. Our chairs, being his patents, embraced and \
caressed us rather than submitted to be sat upon, and there was that \
luxurious after-dinner atmosphere when thought roams gracefully \
free of the trammels of precision. And he put it to us in this \
way--marking the points with a lean forefinger--as we sat and lazily \
admired his earnestness over this new paradox (as we thought it) \
and his fecundity."

In [5]:
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader

# SEU CÓDIGO COM A CLASSE DO DATASET E A FUNÇÃO DO DATA LOADER

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [11]:
# CHAME O DATA LOADER COM TEXTO ACIMA COM A 1ª CONFIGURAÇÃO
# E CONTE OS EXEMPLOS DE TREINO

configs = {
    1: {"batch_size": 4, "max_length": 2, "stride": 1},
    2: {"batch_size": 4, "max_length": 6, "stride": 2},
}


def count_text_examples(text, config):
    dataloader = create_dataloader_v1(
        txt=text,
        batch_size=config["batch_size"],
        max_length=config["max_length"],
        stride=config["stride"],
    )
    tokenizer = tiktoken.get_encoding("gpt2")
    encoded_text = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

    counts = {
        "tokenized": len(encoded_text),
        # "calculated": (len(encoded_text) - config['max_length']) // config['stride'] + 1,
        "dataset": len(dataloader.dataset),
        "train_examples_count": len(dataloader)*config['batch_size'],
        "words": len(text.split()),
        "batches": len(dataloader),
        "tests": len(dataloader.dataset[0]),
    }
    print(f"Para a configuração: {config} o data loader {counts['train_examples_count']} exemplos de treino.")
    for item in dataloader.dataset:
        print(item)
        break
    print(counts)

count_text_examples(time_machine_text, configs[1])


Para a configuração: {'batch_size': 4, 'max_length': 2, 'stride': 1} o data loader 168 exemplos de treino.
(tensor([ 464, 3862]), tensor([ 3862, 43662]))
{'tokenized': 170, 'dataset': 168, 'train_examples_count': 168, 'words': 128, 'batches': 42, 'tests': 2}


In [12]:
# CHAME O DATA LOADER COM TEXTO ACIMA COM A 2ª CONFIGURAÇÃO
# E CONTE OS EXEMPLOS DE TREINO

count_text_examples(time_machine_text, configs[2])

Para a configuração: {'batch_size': 4, 'max_length': 6, 'stride': 2} o data loader 80 exemplos de treino.
(tensor([  464,  3862, 43662,  6051,   357,  1640]), tensor([ 3862, 43662,  6051,   357,  1640,   523]))
{'tokenized': 170, 'dataset': 82, 'train_examples_count': 80, 'words': 128, 'batches': 20, 'tests': 2}


NameError: name 'dataloader' is not defined